In [2]:
import pandas as pd
# from pandas import json_normalize

df = pd.read_json("C:/Users/Gerard/Desktop/VISUAL_STUDIO/PIA/PROYECTO/limpieza/mobiles (1).json")
# df = json_normalize(df.to_dict(orient='records'), sep='_')
# df.columns = [col.replace('detalles_tecnicos_', '') for col in df.columns]
df.set_index('_id', inplace=True)
df

,ASIN,Actualizaciones de software garantizadas hasta,Ano del modelo,Aparatos compatibles,Capacidad de almacenamiento de memoria,Capacidad de almacenamiento digital,Capacidad de la memoria,Capacidad de la memoria RAM,Capacidad de la memoria RAM instalada,Capacidad de la memoria flash instalada,...,Generacion,GPU,Energia de la bateria,Brillo de imagen,Compatibilidad del montaje,Tasa de contraste de imagen,Tipo de fuente de luz,Tipo de montaje,Fabricante del procesador de graficos,Marca de la tarjeta grafica
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,B00IRZ8EQC,desconocido,2013.0,microSD (TransFlash),8 GB,"1,7 Modificador desconocido",8 GB,1536 MB,"1,5 GB","1,5 GB",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00J8OA220,B00J8OA220,desconocido,NaN,NaN,160 GB,NaN,160 GB,NaN,2 GB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00JC8MD7Y,B00JC8MD7Y,NaN,NaN,NaN,6 GB,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TKALUDC,B00TKALUDC,desconocido,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TUXHZTW,B00TUXHZTW,desconocido,2015.0,NaN,12 GB,128 GB,12 GB,NaN,16 GB,128 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,B0DH3N1GZ3,desconocido,NaN,NaN,128 GB,NaN,128 GB,NaN,4 GB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09QH71RZR,B09QH71RZR,NaN,NaN,NaN,128 GB,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0DK4NBZWB,B0DK4NBZWB,desconocido,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Eliminamos las columnas que no aportan valor

In [3]:
columnas_excluidas = {
    "Numero de pilas de ion de litio", 
    "Que hay en la caja?",
    "Formatos de audio compatibles",
    "Otras caracteristicas",
    "Otras caracteristicas",
    "Adaptador",
    "Formatos compatibles", 
    "Tiene estabilizador de imagen", 
    "Tipo de medio", 
    "Aparatos compatibles",
    "Descripcion de la interfaz de red",
    "Caracteristicas especiales",
    "Tecnologia del microfono", 
    "Tecnologia de comunicacion inalámbrica",
    "Grabador",
    "Formatos de audio compatibles",
    "Series",
    "Carga electrica de la bateria",
    "OS",
    "Tiempo de carga",
    "Conector de audio",
    "Resolucion de la camara web trasera",
    "Pilas incluidas",
    "Software incluido",
    "Tamano",
    "Tipo de conector",
    "Ranuras de memoria disponibles",
    "Composicion de la pila de la bateria",
    "Fuente de alimentacion",
    "Proveedor de conexion inalambrica",
    "Velocidad de transferencia de datos", 
    "Descripcion de la bateria", 
    "Duracion de la bateria media (en horas)", 
    "Marca de la tarjeta grafica",
    "Zoom digital",
    "Enfoque automatico",
    "Tiempo de carga de la bateria (en horas)",
    "Memoria extraible",
    "Proveedor de conexion inalambrica",
    "Tamano de la tarjeta de memoria incluida",
    "Relacion de aspecto de imagen", 
    "Dimensiones del paquete", 
    "Embalaje de la bateria de litio", 
    "Tipo de dispositivo", 
    "Actualizaciones de software garantizadas hasta", 
    "Tecnologia de comunicacion inalambrica",
    "Compilador", 
    "Numero de pilas de metal de litio", 
    "Voltaje", 
    "Tiempo de conversacion telefonica", 
    "Numero de puertos USB", 
    "Contenido de energia de la bateria de litio", 
    "Lector", 
    "Marimba", 
    "Tipo de material", 
    "Incluye bateria recargable", 
    "Tiene vision nocturna", 
    "Pantalla a color", 
    "Proveedor de conexion inalámbrica", 
    "Tipo de lente",
    "Descripcion de la tarjeta grafica", 
    "GPS", 
    "Otras caracteristicas de la pantalla", 
    "Tipo de conexion inalámbrica", 
    "Numero de puertos", 
    "Pilas", 
    "Numero de modelo del producto", 
    "Tecnologia de conectividad", 
    "Entrada de interfaz humana", 
    "Otras funciones de la camara", 
    "Color", 
    "Compatible con tecnologia Bluetooth", 
    "0",
    "Referencia del fabricante", 
    "Tipo de conexion inalambrica", 
    "Pilas / baterias incluida", 
    "Pilas / baterias necesarias",
    'Peso del producto',
    'Resolucion del escaner',
    'Numero de producto',
    'Componentes incluidos',
    'Numero de productos',
    'Resolucion del sensor optico',
    'Tecnologia GSM',
    'Factor de forma',
    'Peso de la bateria de litio',
    'Hora de espera del telefono (con datos)',
    'Tecnologia del sintonizador',
    'Contiene liquidos',
    'Duracion media de la pila en stand-by',
    'Transcriptor',
    "Modelo",
    'Duracion media de la bateria',
    'Pilas / baterias incluidas', 
    'Tipo de memoria del ordenador', 
    'Coprocesador grafico', 
    'Nombre del modelo', 
    'Capacidad de la memoria flash instalada',
    'Material', 
    'Sistema operativo', 
    'ASIN', 
    'Operador de servicios inalambricos', 
    'Clasificacion en los mas vendidos de Amazon', 
    'Operador de servicios inalambricos', 
    'Producto en Amazones desde',
    'Restricciones de envio', 
    'titulo',

    'Tecnologia de movil' #??????????????
    

}

df = df.drop(columns=[col for col in columnas_excluidas if col in df.columns])

df

,Ano del modelo,Capacidad de almacenamiento de memoria,Capacidad de almacenamiento digital,Capacidad de la memoria,Capacidad de la memoria RAM,Capacidad de la memoria RAM instalada,Capacidad del disco duro,Capacidad maxima de la memoria RAM,Descripcion del teclado,Dimensiones del producto,...,Interfaz del hardware,Generacion,GPU,Energia de la bateria,Brillo de imagen,Compatibilidad del montaje,Tasa de contraste de imagen,Tipo de fuente de luz,Tipo de montaje,Fabricante del procesador de graficos
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,2013.0,8 GB,"1,7 Modificador desconocido",8 GB,1536 MB,"1,5 GB",8,64 GB,tactil,"10,41 x 5,84 x 1,27 cm; 77,11 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00J8OA220,NaN,160 GB,NaN,160 GB,NaN,2 GB,NaN,NaN,NaN,"25,4 x 22,86 x 10,16 cm; 318 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00JC8MD7Y,NaN,6 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"16,36 x 7,6 x 0,77 cm; 179 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TKALUDC,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,8 x 4,8 x 9,3 cm; 75 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TUXHZTW,2015.0,12 GB,128 GB,12 GB,NaN,16 GB,NaN,NaN,NaN,"4,6 x 1,9 x 9,1 cm; 74 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,NaN,128 GB,NaN,128 GB,NaN,4 GB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09QH71RZR,NaN,128 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7,38 x 0,81 x 15,98 cm; 179 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0DK4NBZWB,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Problemes per solucionar en este codic, si la columna no existeix peta si ja hi ha una columna amb el nom del nom objectiu dins de el diccionari dins de la llista objectiu peta

## Unimos columnas similares

In [4]:
fusion_columnas = {
    "Memoria RAM": ["Tamano de memoria RAM instalada","Capacidad de la memoria RAM instalada", "RAM","Memoria RAM", "Capacidad de la memoria RAM", "Capacidad de la memoria"],
    "Marca": ["Marca", "Fabricante"],
    "Tipo de pantalla": ["Tipo de pantalla", "Tecnologia de pantalla"],
    "Capacidad de almacenamiento digital": ["Capacidad de almacenamiento digital", "Capacidad de almacenamiento de memoria"],
    "Tamano de la pantalla": ["Tamano de pantalla", "Tamano de la pantalla"],
    "Resolucion": ["Resolucion","Maxima resolucion",'Resolucion maxima']


}
for col_final, columnas_equivalentes in fusion_columnas.items():
    columnas_presentes = [col for col in columnas_equivalentes if col in df.columns]

    if columnas_presentes:
        nueva_columna = []
        for index, fila in df.iterrows():
            encontrado = False
            for col in columnas_presentes:
                if pd.notna(fila[col]):
                    nueva_columna.append(fila[col])
                    encontrado = True
                    break
            if not encontrado:
                nueva_columna.append(None)

        df[col_final] = nueva_columna

        for col in columnas_presentes:
            if col != col_final:
                df.drop(columns=col, inplace=True)

In [5]:
df.columns

Index(['Ano del modelo', 'Capacidad de almacenamiento digital',
       'Capacidad del disco duro', 'Capacidad maxima de la memoria RAM',
       'Descripcion del teclado', 'Dimensiones del producto',
       'Fabricante del procesador', 'Incluye mando',
       'Incluye software para emails', 'Marca',
       ...
       'Generacion', 'GPU', 'Energia de la bateria', 'Brillo de imagen',
       'Compatibilidad del montaje', 'Tasa de contraste de imagen',
       'Tipo de fuente de luz', 'Tipo de montaje',
       'Fabricante del procesador de graficos', 'Memoria RAM'],
      dtype='object', length=103)

## Mapeo de columnas

## Hecho con ChatGPT, Falta arreglar para que sea entendible
### {

In [6]:
def convertir_tb_a_gb(valor):
    if pd.notna(valor) and 'TB' in str(valor).upper():
        numero = float(valor.upper().replace('TB', '').strip())
        return numero * 1000
    return valor

In [7]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df['precio_actual'] = df['precio_actual'].replace({',': '', '\\.': ''}, regex=True)
df['precio_actual'] = df['precio_actual'].astype(float)/100

df['precio_anterior'] = df['precio_anterior'].replace({',': '', '\\.': ''}, regex=True)
df['precio_anterior'] = df['precio_anterior'].astype(float)/100

df['Potencia nominal de la bateria'] = df['Potencia nominal de la bateria'].str.extract(r'(\d+)').astype(float)
df['Capacidad de almacenamiento digital'] = df['Capacidad de almacenamiento digital'].apply(convertir_tb_a_gb)
df['Capacidad de almacenamiento digital'] = df['Capacidad de almacenamiento digital'].str.extract(r'(\d+)').astype(float)
df['Resolucion horizontal'] = df['Resolucion horizontal'].str.extract(r'(\d+)').astype(float)
df['Resolucion vertical max'] = df['Resolucion vertical max'].str.extract(r'(\d+)').astype(float)
df['Memoria RAM'] = df['Memoria RAM'].str.extract(r'(\d+)').astype(float)


C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\3611333806.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [8]:
def convertir_tamaño(valor):
    if isinstance(valor, str):
        if 'centimetros' in valor:
            numero = float(valor.split(' ')[0].replace(',', '.'))
            return round(numero / 2.54, 2)
        elif 'pulgadas' in valor:
            numero = float(valor.split(' ')[0].replace(',', '.'))
            return round(numero, 2)
    elif isinstance(valor, (int, float)):
        return round(valor, 2)
    return None

df['Tamano de la pantalla'] = df['Tamano de la pantalla'].apply(convertir_tamaño)

In [9]:
def estandarizar_pantalla(valor):
    if isinstance(valor, str):
        if 'super amoled' in valor or 'superamoled' in valor:
            return 'superamoled'
        elif 'amoled' in valor:
            return 'amoled'
        elif 'oled' in valor:
            return 'oled'
        elif 'lcd' in valor:
            return 'lcd'
    return valor

df['Tipo de pantalla'] = df['Tipo de pantalla'].apply(estandarizar_pantalla)

In [10]:
df['Velocidad de CPU'] = df['Velocidad de CPU'].str.lower().str.strip()

def convertir_a_ghz(valor):
    try:
        # Extraer solo el número y la unidad
        if pd.isna(valor):
            return None

        if 'ghz' in valor:
            return float(valor.replace('ghz', '').replace(',', '.').strip())
        elif 'mhz' in valor:
            return float(valor.replace('mhz', '').replace(',', '.').strip()) / 1000
        elif 'hz' in valor:
            return float(valor.replace('hz', '').replace(',', '.').strip()) / 1_000_000_000
        elif 'e+' in valor:  # Para notación científica
            numero = float(valor)
            if numero >= 1e9:
                return numero / 1e9  # Está en Hz, pasarlo a GHz
            elif numero >= 1e6:
                return numero / 1e6  # Está en MHz, pasarlo a GHz
            elif numero >= 1e3:
                return numero / 1e3  # Está en KHz, pasarlo a GHz
            else:
                return numero / 1_000_000_000  # Está en Hz
        else:
            return None
    except:
        return None

df['Velocidad de CPU (GHz)'] = df['Velocidad de CPU'].apply(convertir_a_ghz)
df = df.drop("Velocidad de CPU", axis=1)


In [11]:
df['valoraciones'] = df['Valoracion media de los clientes'].apply(lambda x: x.split('estrellas', 1)[-1].strip() if isinstance(x, str) and 'estrellas' in x else x)
df = df.drop("Valoracion media de los clientes", axis=1)

In [12]:
import pandas as pd
import re
import numpy as np

def extraer_valoraciones_y_estrellas(texto):
    if pd.isnull(texto):
        return np.nan, np.nan
    
    valoraciones = re.search(r'(\d{1,3}(?:[\.,]?\d{3})*) valoraciones', texto)
    estrellas = re.search(r'(\d[\.,]?\d*) de 5 estrellas', texto)
    
    if valoraciones and estrellas:
        num_valoraciones = valoraciones.group(1).replace(',', '').replace('.', '')
        puntuacion_estrellas = float(estrellas.group(1).replace(',', '.'))
        return int(num_valoraciones), puntuacion_estrellas
    else:
        return np.nan, np.nan

df[['numero_valoraciones', 'numero_estrellas']] = df['valoraciones'].apply(lambda x: pd.Series(extraer_valoraciones_y_estrellas(x)))
df = df.drop("valoraciones", axis=1)

In [13]:
import numpy as np
df['Capacidad de almacenamiento digital'] = df.apply(lambda row: np.nan if row['Capacidad de almacenamiento digital'] == row['Memoria RAM'] else row['Capacidad de almacenamiento digital'], axis=1)

In [14]:
import numpy as np

def corregir_filas(fila):
    if fila['Memoria RAM'] > 36:
        valor_a = fila['Memoria RAM']
        fila['Memoria RAM'] = np.nan
        if pd.notna(fila['Capacidad de almacenamiento digital']) and fila['Capacidad de almacenamiento digital'] < valor_a:
            fila['Memoria RAM'], fila['Capacidad de almacenamiento digital'] = fila['Capacidad de almacenamiento digital'], valor_a
    return fila

df = df.apply(corregir_filas, axis=1)


In [15]:
for index, row in df.iterrows():
    if pd.notna(row['Capacidad de almacenamiento digital']) and row['Capacidad de almacenamiento digital'] < 36:
        if pd.isna(row['Memoria RAM']) or (pd.notna(row['Memoria RAM']) and row['Capacidad de almacenamiento digital'] < row['Memoria RAM']):
            df.at[index, 'Memoria RAM'] = row['Capacidad de almacenamiento digital']
        df.at[index, 'Capacidad de almacenamiento digital'] = np.nan

In [16]:
def extraer_dimensiones_peso(valor):
    try:
        dimensiones, peso = valor.split(';')
        dimensiones = dimensiones.replace('cm', '').strip()
        peso = peso.replace('g', '').replace(',', '.').strip()
        dim_split = [float(d.replace(',', '.').strip()) for d in dimensiones.split('x')]

        if len(dim_split) == 3:
            dim_split.sort()
            return pd.Series([dim_split[2], dim_split[1], dim_split[0], float(peso)])
        else:
            return pd.Series([None, None, None, float(peso)])
    except:
        return pd.Series([None, None, None, None])

df[['Altura (cm)', 'Anchura (cm)', 'Profundidad (cm)', 'Peso (g)']] = df['Dimensiones del producto'].apply(extraer_dimensiones_peso)
df.drop(columns=['Dimensiones del producto'], inplace=True)

C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\3247397040.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[['Altura (cm)', 'Anchura (cm)', 'Profundidad (cm)', 'Peso (g)']] = df['Dimensiones del producto'].apply(extraer_dimensiones_peso)
C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\3247397040.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[['Altura (cm)', 'Anchura (cm)', 'Profundidad (cm)', 'Peso (g)']] = df['Dimensiones del producto'].apply(extraer_dimensiones_peso)
C:\Users\Gerard\AppData\Local\Temp\ipy

In [17]:
import pandas as pd
import re

# Función para limpiar las resoluciones y separarlas en dos columnas (ancho, alto)
def limpiar_resoluciones(df, columna):
    # Expresión regular para identificar resoluciones válidas en formato "Ancho x Alto" o "Ancho*Alto"
    patron = re.compile(r'(\d{3,4})\s*[x*]\s*(\d{3,4})')

    # Crear dos nuevas columnas con el ancho y el alto
    def procesar_resolucion(resolucion):
        if pd.isna(resolucion):
            return None, None
        try:
            resolucion = str(resolucion).lower()
            if any(palabra in resolucion for palabra in ["hd+", "pixels", "~", "fhd+", "p"]):
                return None, None
            match = patron.search(resolucion)
            if match:
                return match.group(1), match.group(2)
            elif "4k" in resolucion:
                return "3840", "2160"
            elif "1080p" in resolucion:
                return "1920", "1080"
            elif "1k" in resolucion:
                return "1024", "768"
        except Exception as e:
            print(f"Error procesando resolución {resolucion}: {e}")
        return None, None

    # Aplicar la función a la columna y separar los resultados en dos nuevas columnas
    df[['ancho', 'alto']] = df[columna].apply(lambda x: pd.Series(procesar_resolucion(x)))

    # Convertir a tipo numérico para análisis posterior (opcional)
    df['ancho'] = pd.to_numeric(df['ancho'], errors='coerce')
    df['alto'] = pd.to_numeric(df['alto'], errors='coerce')

    return df
    # "Resolucion horizontal", 
    # "Resolucion vertical max.", 
df = limpiar_resoluciones(df, "Resolucion")
df = df.drop("Resolucion", axis=1)

C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\599243546.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[['ancho', 'alto']] = df[columna].apply(lambda x: pd.Series(procesar_resolucion(x)))
C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\599243546.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[['ancho', 'alto']] = df[columna].apply(lambda x: pd.Series(procesar_resolucion(x)))


In [18]:
import pandas as pd

def rellenar_alto_ancho(row):
    if pd.isna(row['alto']) and pd.isna(row['ancho']):
        if not pd.isna(row['Resolucion horizontal']) and not pd.isna(row['Resolucion vertical max']):
            row['alto'] = row['Resolucion vertical max']
            row['ancho'] = row['Resolucion horizontal']
    return row

# Suponiendo que df es tu DataFrame
df = df.apply(rellenar_alto_ancho, axis=1)


### }

## Gestion de nulos

In [19]:
df = df.dropna(subset=['precio_actual'])

In [20]:
def rellenar_con_precio(row, df, col_nulos, col_comparar, metodo='moda'):
    if pd.isnull(row[col_nulos]):
        precio = row[col_comparar]
        if pd.notnull(precio):
            rango_min = precio * 0.7
            rango_max = precio * 1.3
            df_filtrado = df[(df[col_comparar] >= rango_min) & (df[col_comparar] <= rango_max)]
            if metodo == 'moda':
                valor = df_filtrado[col_nulos].mode()
                if not valor.empty:
                    return valor[0]
            elif metodo == 'media':
                valor = df_filtrado[col_nulos].mean()
                if not pd.isnull(valor):
                    return valor
    return row[col_nulos]

def rellenar_nulos_con_moda(df, columna):
    moda = df[columna].mode()[0]
    df[columna] = df[columna].fillna(moda)
    return df

def rellenar_nulos_con_media(df, columna):
    media = round(df[columna].mean(), 2)
    df[columna] = df[columna].fillna(media)
    return df


In [21]:
# Capacidad de almacenamiento digital
col_rellenar = "Capacidad de almacenamiento digital"
col_comparar = "precio_actual"
df[col_rellenar] = df.apply(
    lambda row: rellenar_con_precio(row, df, col_rellenar, col_comparar), axis=1
)

# Capacidad de Potencia nominal de la bateria
col_rellenar = "Potencia nominal de la bateria"
df[col_rellenar] = df.apply(
    lambda row: rellenar_con_precio(row, df, col_rellenar, col_comparar), axis=1
)

# Tamano de la pantalla
col_rellenar = "Tamano de la pantalla"
df[col_rellenar] = df.apply(
    lambda row: rellenar_con_precio(row, df, col_rellenar, col_comparar), axis=1
)
# Tipo de pantalla
col_rellenar = "Tipo de pantalla"
df[col_rellenar] = df.apply(
    lambda row: rellenar_con_precio(row, df, col_rellenar, col_comparar), axis=1
)
# Ano del modelo
df = rellenar_nulos_con_moda(df, "Ano del modelo")

# Memoria RAM
col_rellenar = "Memoria RAM"
df[col_rellenar] = df.apply(
    lambda row: rellenar_con_precio(row, df, col_rellenar, col_comparar), axis=1
)
# Profundidad, Peso
df = df.drop(['Altura (cm)', 'Anchura (cm)'], axis=1)
df = rellenar_nulos_con_media(df, "Profundidad (cm)")
df = rellenar_nulos_con_media(df, "Peso (g)")

# Precio anterior
df['precio_anterior'].fillna(df['precio_actual'], inplace=True)

# Valoraciones & Estrellas
col_rellenar = "numero_valoraciones"
df[col_rellenar] = df.apply(
    lambda row: rellenar_con_precio(row, df, col_rellenar, col_comparar), axis=1
)
col_rellenar = "numero_estrellas"
df[col_rellenar] = df.apply(
    lambda row: round(rellenar_con_precio(row, df, col_rellenar, col_comparar, metodo="media"), 1), 
    axis=1
)

df['promedio_valoraciones'] = df['numero_valoraciones'] / df['numero_estrellas']
df = df.drop(["numero_valoraciones", "numero_estrellas"], axis=1)


# Velocidad de CPU (GHz)
col_rellenar = "Velocidad de CPU (GHz)"
df[col_rellenar] = df.apply(
    lambda row: round(rellenar_con_precio(row, df, col_rellenar, col_comparar, metodo="media"), 1), 
    axis=1
)
# Ancho & Alto px
def unir_columnas(df, col1, col2, nueva_col):
    df[nueva_col] = df[col1].dropna().astype(str) + 'x' + df[col2].dropna().astype(str)
    return df
df = unir_columnas(df, 'alto', 'ancho', 'dimensiones')
df_dimension = df['dimensiones']
moda = df_dimension.mode()[0]

moda_altura, moda_anchura = moda.split('x')
moda_altura = float(moda_altura)
moda_anchura = float(moda_anchura)

df['alto'] = df['alto'].fillna(moda_altura)
df['ancho'] = df['ancho'].fillna(moda_anchura)
df = df.drop('dimensiones', axis=1)

C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\121884009.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['precio_anterior'].fillna(df['precio_actual'], inplace=True)
C:\Users\Gerard\AppData\Local\Temp\ipykernel_13916\121884009.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['pro

semisupervisor

Active Learning

## Filtramos columnas
Eliminamos las culomnas que tienen un porcentaje de nulos mayor al indicado

In [22]:
threshold = 0.40 * len(df)
df = df.dropna(axis=1, thresh=threshold)
df

,Ano del modelo,Capacidad de almacenamiento digital,Marca,Tamano de la pantalla,Tipo de pantalla,precio_actual,precio_anterior,Potencia nominal de la bateria,Memoria RAM,Velocidad de CPU (GHz),Profundidad (cm),Peso (g),ancho,alto,promedio_valoraciones
_id,,,,,,,,,,,,,,,
B00IRZ8EQC,2013.0,256.0,samsung,4.30,lcd,245.00,199.99,5000.0,1.0,1.7,1.27,77.11,540.0,960.0,279.459459
B00J8OA220,2024.0,160.0,inmarsat,2.00,lcd,933.00,199.99,160.0,2.0,2.9,10.16,318.00,240.0,320.0,0.800000
B00JC8MD7Y,2024.0,256.0,samsung,6.70,superamoled,279.99,279.99,4500.0,6.0,2.2,0.77,179.00,1080.0,2400.0,1761.818182
B00TKALUDC,2015.0,128.0,ttfone,2.00,lcd,41.99,45.99,800.0,32.0,0.0,1.80,75.00,1080.0,2400.0,168.571429
B00TUXHZTW,2015.0,128.0,doro,2.00,lcd,48.67,48.67,800.0,16.0,1.2,1.90,74.00,3840.0,2160.0,74.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,2024.0,128.0,honor,6.56,lcd,99.90,199.99,5000.0,4.0,22.1,2.57,254.42,1920.0,1080.0,1.702128
B09QH71RZR,2024.0,128.0,xiaomi,6.43,amoled,127.00,165.39,5000.0,6.0,2.0,0.81,179.00,1920.0,1080.0,3821.818182
B0DK4NBZWB,2024.0,256.0,oppo,6.67,amoled,299.00,199.99,5000.0,8.0,3.4,2.57,254.42,1080.0,2400.0,2.291667


In [23]:
# Valorem de eliminar els nuls que no s'han pogut eliminar per el nosetre criteri
print(f'{df.dropna().shape[0]}/{df.shape[0]}')

941/953


In [24]:
df = df.dropna()

## Refactoritcació de columnes

In [25]:
df = df.rename(
    columns={
        'Ano del modelo': 'ano', 
        'Capacidad de almacenamiento digital': 'memoria',
        'Tamano de la pantalla':'pantalla_in',
        'Tipo de pantalla':'pantalla_tipo',
        'Potencia nominal de la bateria':'bateria',
        'Memoria RAM':'ram',
        'Profundidad (cm)':'grosor',
        'Peso (g)':'peso',
        'ancho':'ancho_px',
        'alto':'alto_px',
        'Velocidad de CPU (GHz)':'velocidad_cpu_ghz'
        }
    )


df.columns = df.columns.str.lower()
nuevo_orden = [
    'ano', 'memoria', 'marca',
    'pantalla_in', 'pantalla_tipo',
    'velocidad_cpu_ghz', 'ram',
    'grosor', 'peso', 'ancho_px', 'alto_px',
    'bateria',                         
    'promedio_valoraciones',
    'precio_anterior', 'precio_actual'
]
df = df[nuevo_orden]

## Deteccion de Outliders

In [33]:
def eliminar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    filtro = (df[columna] >= (Q1 - 1.5 * IQR)) & (df[columna] <= (Q3 + 1.5 * IQR))
    return df[filtro]

def detectar_outlier_precio(df):
    return eliminar_outliers(df, 'precio')

def detectar_outlier_ram(df):
    return eliminar_outliers(df, 'ram')

def detectar_outlier_almacenamiento(df):
    return eliminar_outliers(df, 'almacenamiento')

def detectar_outlier_precio_anterior(df):
    return eliminar_outliers(df, 'precio_anterior')

def detectar_outlier_bateria(df):
    return eliminar_outliers(df, 'bateria')

def detectar_outlier_valoraciones(df):
    return eliminar_outliers(df, 'promedio_valoraciones')

def detectar_outlier_cpu(df):
    return eliminar_outliers(df, 'velocidad_cpu_ghz')

def detectar_outlier_anos(df):
    return eliminar_outliers(df, 'ano')

def detectar_outlier_grosor(df):
    return eliminar_outliers(df, 'grosor')

In [34]:
def procesar_dataframe(df):
    df = detectar_outlier_precio(df)
    df = detectar_outlier_ram(df)
    df = detectar_outlier_almacenamiento(df)
    df = detectar_outlier_precio_anterior(df)
    df = detectar_outlier_bateria(df)
    df = detectar_outlier_valoraciones(df)
    df = detectar_outlier_cpu(df)
    df = detectar_outlier_anos(df)
    df = detectar_outlier_grosor(df)
    return df

In [35]:
df.to_csv("temp.csv")

### Insertar MongoDB

In [29]:
from pymongo import MongoClient

# Conexión a MongoDB
cliente = MongoClient("mongodb://localhost:27020/")
db = cliente["PIA"]
coleccion = db["products_cleaned"]

df_reset = df.reset_index()
data_dict = df_reset.to_dict("records")

for data in data_dict:
    coleccion.update_one(
        {'_id': data['_id']},
        {'$set': data},
        upsert=True
    )

print("Datos insertados o actualizados")


ServerSelectionTimeoutError: localhost:27020: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67b8631f63033aa2b98e46dd, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27020) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27020: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

### Exportar MongoDB

In [341]:
from datetime import datetime
import json

fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
carpeta = "../backup/"
nombre_archivo = f"backup_mobiles_cleaned_{fecha_hora_actual}.json"
destino = carpeta + nombre_archivo
documentos = list(coleccion.find())

for doc in documentos:
    doc['_id'] = str(doc['_id'])

with open(destino, "w", encoding="utf-8") as archivo:
    json.dump(documentos, archivo, ensure_ascii=False, indent=4)

print("Base de datos descargada con éxito")

Base de datos descargada con éxito
